In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_google_genai import ChatGoogleGenerativeAI
import getpass
import os
import pandas as pd

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass('enter google api key here')

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
file_path = ('enter file locaiton here')
data = pd.read_csv(file_path, encoding='unicode_escape')

In [3]:
data.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,CUSTOMERNAME,COUNTRY,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,Shipped,1,2,2003,Motorcycles,Land of Toys Inc.,USA,Kwai,Small
1,10121,34,81.35,5,2765.90,Shipped,2,5,2003,Motorcycles,Reims Collectables,France,Paul,Small
2,10134,41,94.74,2,3884.34,Shipped,3,7,2003,Motorcycles,Lyon Souveniers,France,Daniel,Medium
3,10145,45,83.26,6,3746.70,Shipped,3,8,2003,Motorcycles,Toys4GrownUps.com,USA,Julie,Medium
4,10159,49,100.00,14,5205.27,Shipped,4,10,2003,Motorcycles,Corporate Gift Ideas Co.,USA,Julie,Medium


In [4]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [5]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS


In [6]:
#embeddings are created here
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
index = faiss.IndexFlatL2(len(embeddings.embed_query(" ")))


In [7]:
vector_store = FAISS(
    embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [8]:
vector_store.add_documents(documents=docs)

['5e2f6981-2f4a-49ea-89b3-c2c1ac241f84',
 '4752aa99-468b-4d07-9097-566bec19cc26',
 '82fe82ac-d88c-469a-b47a-344513ebb242',
 '6e923829-1ee3-4f41-8eef-ab17c1c2d3c5',
 'a7ce4286-cf91-430d-b316-b4997a950ed1',
 '75a77b3f-fbc5-40e6-86c6-7fc0eedd1e43',
 '82918ba3-2b3a-4021-9f6d-435dac90343c',
 '9ba754f4-44bd-49d3-bb58-f9b405b44d3b',
 'ddcbb48c-7557-48ac-85ad-29479540d933',
 '09e31f99-6edc-4014-bc4e-cd9767f2371b',
 'b5e23263-372e-4fcb-b1d8-786ef3ae1d0a',
 'd1a5a626-1dd8-4911-85bd-850af24f7c8e',
 '5567316b-bb5d-44fe-bb1b-8f17431bf61d',
 'e7d71efe-5490-4803-b707-2ac420075dcd',
 'c45dc780-95ae-424c-9c67-a1246df1534d',
 'e72c3ef9-a06b-4494-88b0-f70f428a7060',
 '8a1e74e3-3812-4447-81d9-4c7c2179d25d',
 '6c212b51-87dd-4066-82c7-1215411a090c',
 'eaf37478-7664-48e2-b8ba-6c60eae72cf8',
 'de3ca2a1-910c-4e5d-8f14-a675d4406efe',
 '0f0dff53-878b-4a47-bc59-2c7e7a06fbb5',
 '2fcb8228-e2ca-4a1d-a71e-1bd867e69fa8',
 'c35e3b7a-2514-4a66-afb3-b59b8e9886a6',
 '91934ef9-9a7d-48f9-a313-b4364016e133',
 '49fb4125-6af8-

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import sqlite3


retriever = vector_store.as_retriever()

system_prompt = (
    "You are an expert in answering questions based on data from a CSV file named 'sales_data_sample'. "
    "The CSV file has the following columns: "
    "ORDERNUMBER, QUANTITYORDERED, ORDERLINENUMBER, QTR_ID, MONTH_ID, YEAR_ID, PRICEEACH, SALES, STATUS, CONTACTFIRSTNAME, "
    "COUNTRY, DEALSIZE, PRODUCTLINE. "
    "For example: "
    "Example 1 - How many orders have been placed?, "
    "the answer will be the count of orders: 1000 "
    "Example 2 - Show all orders from customers in the USA, "
    "the answer will be the list of orders for customers in the USA: "
    "[Order1, Order2, Order3] "
    "You should directly return the result of the query, not the SQL query itself. "
    "If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"), 
])

conn = sqlite3.connect(":memory:") 
data.to_sql('sales_data_sample', conn, index=False, if_exists='replace')

# Function to execute the SQL query 
def execute_sql_query(query):
    try:
        result = pd.read_sql_query(query, conn)
        return result
    except Exception as e:
        return str(e)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
answer = rag_chain.invoke({"input": "The customerfirstname Veysel with customername Herkku Gifts from the country Norway with sales 5512.32 has what productline in year 2003?"})
result = answer['answer']  
print(result)

The customer has the productline Motorcycles.  This is based on the provided data where Veysel from Herkku Gifts in Norway with sales 5512.32 in 2003 purchased Motorcycles.  The order was placed in Q4, specifically November.
